In [1]:
import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from gauge_utils import SimpleProblem

import gurobipy as gp
from gurobipy import Model, GRB, QuadExpr, LinExpr

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

num_var = 10
num_ineq = 5
num_eq = 5
num_examples = 10
print(num_ineq, num_eq)
np.random.seed(17)
Q = np.diag(np.random.random(num_var))
p = np.random.random(num_var)
A = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
X = np.random.uniform(-1, 1, size=(num_examples, num_eq))
G = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))
h = np.sum(np.abs(G@np.linalg.pinv(A)), axis=1)
L = np.ones((num_var))*-5
U = np.ones((num_var))*5
data = SimpleProblem(Q, p, A, G, h, X, L, U)
data.calc_Y()
print(len(data.Y))
data.remove_no_ip()
print(len(data.Y))


5 5
running osqp
10


In [2]:
[data.L_np[i] for i in data.partial_vars]

[-5.0, -5.0, -5.0, -5.0, -5.0]

In [3]:
def get_interior_point(data, x): 
    tmp_h = data.h_np - data.G_np[:, data.other_vars] @ data._A_other_inv.detach().cpu().numpy() @ x
    tmp_G = data.G_np[:, data.partial_vars] - data.G_np[:, data.other_vars] @ data._A_other_inv.detach().cpu().numpy() @ data.A_np[:, data.partial_vars]
    M = 1e4
    # lb_p = np.array([data.L_np[i] for i in data.partial_vars])
    # ub_p = np.array([data.U_np[i] for i in data.partial_vars])
    
    model = gp.Model("interior_point")
    model.setParam('OutputFlag', 1)

    n_vars = len(data.partial_vars)
    vars = []
    for i in range(n_vars):
        # vars.append(model.addVar(lb=lb_p[i],
        #                          ub=ub_p[i], vtype=GRB.CONTINUOUS, name=f"yp_{i}"))
        vars.append(model.addVar(lb=-np.infty,
                                 ub=np.infty, vtype=GRB.CONTINUOUS, name=f"yp_{i}"))
    
    ya = model.addVar(lb=-np.infty, ub=0, vtype=GRB.CONTINUOUS, name="ya")

    obj = LinExpr()

    obj.add(M * ya)
    model.setObjective(obj, GRB.MINIMIZE)

    for i in range(tmp_G.shape[0]):
        expr = LinExpr()
        for j in range(n_vars):
            if tmp_G[i, j] != 0:
                expr.add(vars[j] * tmp_G[i, j])
        model.addConstr(expr - ya <= tmp_h[i])
    
    return model

In [4]:
m = get_interior_point(data, data.X_np[3])

In [5]:
m.getObjective()

<gurobi.LinExpr: 0.0>

In [6]:
for c in m.getConstrs():
    print(c.getAttr('RHS'))

In [7]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 25 rows, 6 columns and 110 nonzeros
Model fingerprint: 0x6dabf2db
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e+04, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 7e+00]
Presolve time: 0.00s
Presolved: 25 rows, 6 columns, 110 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
      13   -2.8093174e+04   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.00 seconds (0.00 work units)
Optimal objective -2.809317389e+04


In [12]:
v = torch.randn((data.X.shape[0], len(data.partial_vars)))
v.shape

torch.Size([4, 50])

In [4]:
g_unit_ball = data.gauge_unit_ball(v)
g_unit_ball.shape

torch.Size([4])

In [5]:
g_set = data.gauge_set(v, data.IP, data.X)
g_set.shape

torch.Size([4])

In [6]:
v = torch.randn((data.X.shape[0], len(data.partial_vars)))

In [10]:
((g_unit_ball / g_set).unsqueeze(1) * v + data.IP).shape

torch.Size([4, 50])

In [8]:
g_unit_ball / g_set

tensor([0.0242, 0.1574, 0.2101, 0.4979])